# This file details the step by step analysis of the russian asat

In [1]:
from pathlib import Path
HERE = Path('.').resolve()
OUTPUT = HERE / 'output' / 'russian_asat'
OUTPUT.mkdir(exist_ok=True)
print(f'Using {OUTPUT} as output')

Using /home/danielk/git/antisat_analysis/projects/output/russian_asat as output


In [2]:
import sys
print(f'Adding {str(HERE.parent)} to path')
sys.path.append(str(HERE.parent))

Adding /home/danielk/git/antisat_analysis to path


In [3]:
spade_data_paths = {
    'uhf': [
        '/home/danielk/data/spade/beamparks/uhf/2021.11.23',
        '/home/danielk/data/spade/beamparks/uhf/2021.11.25',
        '/home/danielk/data/spade/beamparks/uhf/2021.11.29',
    ],
    'esr': [
        '/home/danielk/data/spade/beamparks/esr/2021.11.19',
        '/home/danielk/data/spade/beamparks/esr/2021.11.23',
        '/home/danielk/data/spade/beamparks/esr/2021.11.25',
        '/home/danielk/data/spade/beamparks/esr/2021.11.29',
    ]
}

data_paths = {}
for key in spade_data_paths:
    data_paths[key] = []
    for pth in spade_data_paths[key]:
        data_paths[key].append(OUTPUT / key / (Path(pth).name + '.h5'))

## Convert event.txt to h5

In [4]:
from convert_spade_events_to_h5 import main as conv_main

for key in spade_data_paths:
    for pth, out in zip(spade_data_paths[key], data_paths[key]):

        pth = Path(pth).resolve()
        if not out.parent.is_dir():
            out.parent.mkdir(parents=True, exist_ok=True)
        if not out.is_file():
            cmd = f'{str(pth)} {str(out)}'
            print(f'Running {cmd}')
            conv_main(cmd.split())
        else:
            print(f'{out} exsists, skipping...')


/home/danielk/git/antisat_analysis/projects/output/russian_asat/uhf/2021.11.23.h5 exsists, skipping...
/home/danielk/git/antisat_analysis/projects/output/russian_asat/uhf/2021.11.25.h5 exsists, skipping...
/home/danielk/git/antisat_analysis/projects/output/russian_asat/uhf/2021.11.29.h5 exsists, skipping...
/home/danielk/git/antisat_analysis/projects/output/russian_asat/esr/2021.11.19.h5 exsists, skipping...
/home/danielk/git/antisat_analysis/projects/output/russian_asat/esr/2021.11.23.h5 exsists, skipping...
/home/danielk/git/antisat_analysis/projects/output/russian_asat/esr/2021.11.25.h5 exsists, skipping...
/home/danielk/git/antisat_analysis/projects/output/russian_asat/esr/2021.11.29.h5 exsists, skipping...


In [5]:
from astropy.time import Time, TimeDelta
import h5py
import numpy as np

# Determine dates of files
dates = []
dates_keyed = {}
for key in data_paths:
    dates_keyed[key] = []
    for pth in data_paths[key]:
        with h5py.File(pth, 'r') as hf:
            epoch = Time(np.min(hf['t']), format='unix', scale='utc')
            dates.append(epoch.datetime.date())
            dates_keyed[key].append(epoch.datetime.date())

dates = np.unique(dates).tolist()

In [6]:
for dt in dates:
    print(dt.strftime('%Y-%m-%d'))

2021-11-19
2021-11-23
2021-11-25
2021-11-29


## Download spacetrack catalog

Run the commands generate below to get the appropriate catalogs

In [7]:
# Set this to None if you dont want to use a secret-tool to get credentials
#  they need to be stored under the "password" and "username" attributes
secret_tool_key = 'space-track'

catalog_commands = []
tle_catalogs = []
for dt in dates:
    dt_str = dt.strftime('%Y-%m-%d')
    
    out_tle = OUTPUT / f'{dt_str.replace("-","_")}_spacetrack.tle'
    tle_catalogs.append(out_tle)
    
    cmd = f'./space_track_download.py 24h {dt_str} {str(out_tle)}'
    if secret_tool_key is not None:
        cmd += f' -k {secret_tool_key}'
    
    print(cmd)
    catalog_commands.append(cmd)

./space_track_download.py 24h 2021-11-19 /home/danielk/git/antisat_analysis/projects/output/russian_asat/2021_11_19_spacetrack.tle -k space-track
./space_track_download.py 24h 2021-11-23 /home/danielk/git/antisat_analysis/projects/output/russian_asat/2021_11_23_spacetrack.tle -k space-track
./space_track_download.py 24h 2021-11-25 /home/danielk/git/antisat_analysis/projects/output/russian_asat/2021_11_25_spacetrack.tle -k space-track
./space_track_download.py 24h 2021-11-29 /home/danielk/git/antisat_analysis/projects/output/russian_asat/2021_11_29_spacetrack.tle -k space-track


In [8]:
#Verify that catalogs exist
for pth in tle_catalogs:
    assert pth.is_file(), f'{pth} does not exist'
    print(f'{pth} exists...')

/home/danielk/git/antisat_analysis/projects/output/russian_asat/2021_11_19_spacetrack.tle exists...
/home/danielk/git/antisat_analysis/projects/output/russian_asat/2021_11_23_spacetrack.tle exists...
/home/danielk/git/antisat_analysis/projects/output/russian_asat/2021_11_25_spacetrack.tle exists...
/home/danielk/git/antisat_analysis/projects/output/russian_asat/2021_11_29_spacetrack.tle exists...


## Correlate measurnments

### Calculate correlations

Choose number of cores to run on

In [9]:
cores = 6
override = False
jitter = True
r_scaling = 1.0
dr_scaling = 0.2
save_states = False

In [13]:
correlate_commands = []

override_cm = '-c' if override else ''
jitter_cm = '--jitter' if jitter else ''
r_scaling_cm = f'--range-scaling {r_scaling}'
dr_scaling_cm = f'--range-rate-scaling {dr_scaling}'

correlations = {}
for key in data_paths:
    correlations[key] = []
    for pth, dt in zip(data_paths[key], dates_keyed[key]):
        tle_file = tle_catalogs[dates.index(dt)]
        out_file = OUTPUT / (pth.stem + '_correlation.pickle')
        
        correlations[key].append(out_file)
        
        cmd = f'mpirun -n {cores} ./beampark_correlator.py eiscat_{key} '
        cmd += f'{str(tle_file)} {str(pth)} {str(out_file)} {override_cm} '
        cmd += f'{jitter_cm} {r_scaling_cm} {dr_scaling_cm} -c'
        print(cmd + '\n')
        correlate_commands.append(cmd)



mpirun -n 6 ./beampark_correlator.py eiscat_uhf /home/danielk/git/antisat_analysis/projects/output/russian_asat/2021_11_23_spacetrack.tle /home/danielk/git/antisat_analysis/projects/output/russian_asat/uhf/2021.11.23.h5 /home/danielk/git/antisat_analysis/projects/output/russian_asat/2021.11.23_correlation.pickle  --jitter --range-scaling 1.0 --range-rate-scaling 0.2 -c

mpirun -n 6 ./beampark_correlator.py eiscat_uhf /home/danielk/git/antisat_analysis/projects/output/russian_asat/2021_11_25_spacetrack.tle /home/danielk/git/antisat_analysis/projects/output/russian_asat/uhf/2021.11.25.h5 /home/danielk/git/antisat_analysis/projects/output/russian_asat/2021.11.25_correlation.pickle  --jitter --range-scaling 1.0 --range-rate-scaling 0.2 -c

mpirun -n 6 ./beampark_correlator.py eiscat_uhf /home/danielk/git/antisat_analysis/projects/output/russian_asat/2021_11_29_spacetrack.tle /home/danielk/git/antisat_analysis/projects/output/russian_asat/uhf/2021.11.29.h5 /home/danielk/git/antisat_analysis

In [12]:
#Verify that catalogs exist
for key in correlations:
    for pth in correlations[key]:
        assert pth.is_file(), f'{pth} does not exist'
        print(f'{pth} exists...')

/home/danielk/git/antisat_analysis/projects/output/russian_asat/2021.11.23_correlation.pickle exists...
/home/danielk/git/antisat_analysis/projects/output/russian_asat/2021.11.25_correlation.pickle exists...


AssertionError: /home/danielk/git/antisat_analysis/projects/output/russian_asat/2021.11.29_correlation.pickle does not exist

### Analyse correalations